In [1]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
import tree
import pickle
import numpy as np

In [3]:
tt = tree.tmtree.Tree(is_gal=True)
#tt.dump()

#tt = pickle.load(open("tree_meta_gal.pickle", "rb"))
#tt.load_np()

In [43]:
tt.fatherID = tt.fatherID[1:]
tt.fatherIDx= tt.fatherIDx[1:]

In [155]:
tnow = tt.tree[tt.tree["nstep"]==max(tt.tree["nstep"])]
large_last = tnow[(tnow["m"] > 1) * (tnow["m"] < 2)]
large_last["idx"][24]

7864800

In [154]:
tt.tree[7864648]["id"]

17

In [156]:
maintree, idx_prgs_alltime2 = extract_direct_full_tree(tt,7864800)
all_direct_prgs2 = get_all_trees(tt,idx_prgs_alltime2)

0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
53-th step
54-th step
55-th step
56-th step
57-th step
58-th step
59-th step
60-th step


In [157]:
from matplotlib import cm
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


for i, gals_this_step in enumerate(all_direct_prgs2):
    for  this_gal in gals_this_step:
        if this_gal is None:
            continue
        ax.plot(this_gal["xp"][:,0],this_gal["xp"][:,1],this_gal["xp"][:,2],
                c=cm.hot(i*4))

plt.legend()
plt.show()

/home/hoseung/Work/hspy/lib/python3.5/site-packages/matplotlib/axes/_axes.py:545: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [158]:
plt.close()

In [159]:
all_data = pickle.dump(all_direct_prgs2,open("all_direct_prgs_data.pickle","wb"))

In [141]:
def extract_direct_full_tree(self, idx):
    """
    Extracts main progenitors from a TreeMaker tree.


    example
    -------
    >>> tt = tmtree.Tree("tree.dat")
    >>> atree = tt.extract_main_tree(12345)

    TODO
    ----
    It works, but the try - except clause is error-prone.
    Explicitly check the end of progenitor tree and make the function more predictable.

    """

    t = self.tree
    #fatherID = self.fatherID
    fatherIDx = self.fatherIDx
    fatherMass = self.fatherMass

    t_now = t[idx]
    nstep = t_now["nstep"]
    nouts = [nstep]
    atree = np.zeros(nstep + 1, dtype=t.dtype)
    atree[0] = t_now

    idx_prgs_alltime = []

    for i in range(1, nstep + 1):
        try:
            idx_father = fatherIDx[t["f_ind"][idx]:t["f_ind"][idx]+t["nprgs"][idx]] -1
            if len(idx_father) > 0:
                idx_prgs_alltime.append(idx_father[idx_father>0])
                mass_father = fatherMass[t["f_ind"][idx]:t["f_ind"][idx]+t["nprgs"][idx]]
                idx = idx_father[np.argmax(mass_father)]
                if idx < 1:
                    break
                t_father=t[idx]
                atree[i]=t_father
                nouts.append(nstep)
            else:
                break
        except:
            break

    return atree, idx_prgs_alltime

def get_all_trees(t,idx_prgs_alltime):
    all_main_prgs=[]
    # Per nstep
    for j, satellie_roots in enumerate(idx_prgs_alltime):
        print("{}-th step".format(j))
        mainprgs=[]
        # Per galaxy
        for sat in satellie_roots:
            mainprgs.append(extract_main_tree(t,sat))
        all_main_prgs.append(mainprgs)
            
    return all_main_prgs


In [123]:
def extract_main_tree(self, idx):
    """
    Extracts main progenitors from a TreeMaker tree.


    example
    -------
    >>> tt = tmtree.Tree("tree.dat")
    >>> atree = tt.extract_main_tree(12345)

    TODO
    ----
    It works, but the try - except clause is error-prone.
    Explicitly check the end of progenitor tree and make the function more predictable.

    """

    t = self.tree
    fatherIDx = self.fatherIDx
    fatherMass = self.fatherMass

    t_now = t[idx]
    nstep = t_now["nstep"]
    nouts = [nstep]
    atree = np.zeros(nstep + 1, dtype=t.dtype)
    atree[0] = t_now

    if nstep <= 1:
        return

    for i in range(1, nstep + 1):
        try:
            idx_father = fatherIDx[t["f_ind"][idx]:t["f_ind"][idx]+t["nprgs"][idx]]
            if len(idx_father) > 0:
                mass_father = fatherMass[t["f_ind"][idx]:t["f_ind"][idx]+t["nprgs"][idx]]
                idx = idx_father[np.argmax(mass_father)] -1
                if idx < 1:
                    break
                t_father=t[idx]
                #idx = t_father["idx"]
                atree[i]=t_father
                nouts.append(nstep)
            else:
                break
        except:
            break

    return np.copy(atree[:i])

In [ ]:
def get_prgs(self,idx):
    t = self.tree
    fatherIDx = self.fatherIDx
    fatherMass = self.fatherMass
    
    idx_father = fatherIDx[t["f_ind"][idx]:t["f_ind"][idx]+t["nprgs"][idx]]
    if len(id_father) > 0:
        #mass_father = fatherMass[t["f_ind"][idx]:t["f_ind"][idx]+t["nprgs"][idx]]

        t_father = t[idx_father]
        idx = t_father["idx"][0] # need to be a integer scalar.
        atree.append(t_father)
        #nouts.append(nstep)
    else:
        break

In [ ]:
# sample galaxies at z=0



# find full merger tree of the sample galaxies